# Loop 42 Analysis: ChemBERTa Results and Strategic Assessment

**Key Question:** What can we learn from the ChemBERTa experiment and what should we try next?

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# Submission history
submissions = [
    ('exp_000', 0.011081, 0.098160),
    ('exp_001', 0.012297, 0.106490),
    ('exp_003', 0.010501, 0.097190),
    ('exp_005', 0.010430, 0.096910),
    ('exp_006', 0.009749, 0.094570),
    ('exp_007', 0.009262, 0.093160),
    ('exp_009', 0.009192, 0.093640),
    ('exp_012', 0.009004, 0.091340),
    ('exp_024', 0.008689, 0.089290),
    ('exp_026', 0.008465, 0.088750),
    ('exp_030', 0.008298, 0.087720),
    ('exp_035', 0.009825, 0.096960),
]

cv_scores = np.array([s[1] for s in submissions])
lb_scores = np.array([s[2] for s in submissions])

# Linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(cv_scores, lb_scores)
print(f'CV-LB Relationship: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'R-squared = {r_value**2:.4f}')
print(f'\nTarget LB: 0.0347')
print(f'Intercept: {intercept:.4f}')
print(f'\nTo reach target LB = 0.0347:')
required_cv = (0.0347 - intercept) / slope
print(f'Required CV = {required_cv:.6f}')

CV-LB Relationship: LB = 4.29 * CV + 0.0528
R-squared = 0.9523

Target LB: 0.0347
Intercept: 0.0528

To reach target LB = 0.0347:
Required CV = -0.004218


In [2]:
# Analyze the gap
print('=== CV-LB Gap Analysis ===')
print()
for exp_id, cv, lb in submissions:
    gap = lb - cv
    ratio = lb / cv
    print(f'{exp_id}: CV={cv:.6f}, LB={lb:.6f}, Gap={gap:.4f}, Ratio={ratio:.1f}x')

print(f'\nAverage gap: {np.mean(lb_scores - cv_scores):.4f}')
print(f'Average ratio: {np.mean(lb_scores / cv_scores):.1f}x')

=== CV-LB Gap Analysis ===

exp_000: CV=0.011081, LB=0.098160, Gap=0.0871, Ratio=8.9x
exp_001: CV=0.012297, LB=0.106490, Gap=0.0942, Ratio=8.7x
exp_003: CV=0.010501, LB=0.097190, Gap=0.0867, Ratio=9.3x
exp_005: CV=0.010430, LB=0.096910, Gap=0.0865, Ratio=9.3x
exp_006: CV=0.009749, LB=0.094570, Gap=0.0848, Ratio=9.7x
exp_007: CV=0.009262, LB=0.093160, Gap=0.0839, Ratio=10.1x
exp_009: CV=0.009192, LB=0.093640, Gap=0.0844, Ratio=10.2x
exp_012: CV=0.009004, LB=0.091340, Gap=0.0823, Ratio=10.1x
exp_024: CV=0.008689, LB=0.089290, Gap=0.0806, Ratio=10.3x
exp_026: CV=0.008465, LB=0.088750, Gap=0.0803, Ratio=10.5x
exp_030: CV=0.008298, LB=0.087720, Gap=0.0794, Ratio=10.6x
exp_035: CV=0.009825, LB=0.096960, Gap=0.0871, Ratio=9.9x

Average gap: 0.0848
Average ratio: 9.8x


In [3]:
# ChemBERTa experiment results
print('=== ChemBERTa Experiment Results (exp_041) ===')
print()
print('Single fold (1,1,1,3,3,3-Hexafluoropropan-2-ol):')
print('  ChemBERTa only (768-dim): MSE = 0.061135')
print('  ChemBERTa PCA (20-dim): MSE = 0.058003')
print('  ChemBERTa + Spange: MSE = 0.042313')
print('  ChemBERTa PCA + Spange: MSE = 0.041895')
print()
print('  Spange only: MSE = 0.034241')
print('  DRFP only: MSE = 0.059665')
print('  Spange + DRFP: MSE = 0.057212')
print()
print('Full CV (ChemBERTa PCA + Spange):')
print('  Mean MSE: 0.010288 +/- 0.008427')
print('  Baseline (exp_035): CV = 0.008194')
print()
print('CONCLUSION: ChemBERTa is 25.5% WORSE than baseline')

=== ChemBERTa Experiment Results (exp_041) ===

Single fold (1,1,1,3,3,3-Hexafluoropropan-2-ol):
  ChemBERTa only (768-dim): MSE = 0.061135
  ChemBERTa PCA (20-dim): MSE = 0.058003
  ChemBERTa + Spange: MSE = 0.042313
  ChemBERTa PCA + Spange: MSE = 0.041895

  Spange only: MSE = 0.034241
  DRFP only: MSE = 0.059665
  Spange + DRFP: MSE = 0.057212

Full CV (ChemBERTa PCA + Spange):
  Mean MSE: 0.010288 +/- 0.008427
  Baseline (exp_035): CV = 0.008194

CONCLUSION: ChemBERTa is 25.5% WORSE than baseline


In [4]:
# What have we tried that DIDN'T work?
print('=== Approaches That FAILED ===')
print()
failed_approaches = [
    ('DRFP with PCA (exp_002)', 0.016948, 'DRFP alone loses information'),
    ('Deep Residual MLP (exp_004)', 0.051912, 'Too complex for small data'),
    ('Minimal features (exp_038)', 0.009825, 'Need DRFP features'),
    ('GNN (exp_040)', 0.068767, 'Single fold only - very poor'),
    ('ChemBERTa (exp_041)', 0.010288, 'Pre-trained embeddings not helpful'),
    ('k-NN with Tanimoto', 0.072666, 'Single fold only - very poor'),
]

for name, score, reason in failed_approaches:
    print(f'{name}: CV = {score:.6f}')
    print(f'  Reason: {reason}')
    print()

=== Approaches That FAILED ===

DRFP with PCA (exp_002): CV = 0.016948
  Reason: DRFP alone loses information

Deep Residual MLP (exp_004): CV = 0.051912
  Reason: Too complex for small data

Minimal features (exp_038): CV = 0.009825
  Reason: Need DRFP features

GNN (exp_040): CV = 0.068767
  Reason: Single fold only - very poor

ChemBERTa (exp_041): CV = 0.010288
  Reason: Pre-trained embeddings not helpful

k-NN with Tanimoto: CV = 0.072666
  Reason: Single fold only - very poor



In [5]:
# What HAS worked?
print('=== Approaches That WORKED ===')
print()
working_approaches = [
    ('Spange descriptors', 'Physicochemical properties relevant to solvation'),
    ('DRFP features (high-variance)', 'Molecular structure fingerprints'),
    ('Arrhenius kinetics', 'Physics-informed features'),
    ('GP component', 'Different inductive bias, helps with uncertainty'),
    ('MLP + LGBM ensemble', 'Complementary model types'),
    ('Weighted loss [1,1,2] for SM', 'SM is harder to predict'),
    ('ACS PCA features', 'Additional chemical descriptors'),
]

for name, reason in working_approaches:
    print(f'{name}')
    print(f'  Why: {reason}')
    print()

=== Approaches That WORKED ===

Spange descriptors
  Why: Physicochemical properties relevant to solvation

DRFP features (high-variance)
  Why: Molecular structure fingerprints

Arrhenius kinetics
  Why: Physics-informed features

GP component
  Why: Different inductive bias, helps with uncertainty

MLP + LGBM ensemble
  Why: Complementary model types

Weighted loss [1,1,2] for SM
  Why: SM is harder to predict

ACS PCA features
  Why: Additional chemical descriptors



In [6]:
# The critical insight: CV-LB intercept problem
print('=== THE CRITICAL PROBLEM ===')
print()
print(f'CV-LB relationship: LB = {slope:.2f} * CV + {intercept:.4f}')
print(f'Target LB: 0.0347')
print(f'Intercept: {intercept:.4f}')
print()
print('PROBLEM: Intercept (0.0524) > Target (0.0347)')
print('This means even with CV = 0, predicted LB would be 0.0524')
print()
print('IMPLICATION: We need to CHANGE the CV-LB relationship, not just improve CV')
print()
print('Possible causes of high intercept:')
print('1. Systematic overfitting to training solvents')
print('2. Mismatch between local CV and Kaggle evaluation')
print('3. Distribution shift between train/test')
print('4. Model miscalibration')

=== THE CRITICAL PROBLEM ===

CV-LB relationship: LB = 4.29 * CV + 0.0528
Target LB: 0.0347
Intercept: 0.0528

PROBLEM: Intercept (0.0524) > Target (0.0347)
This means even with CV = 0, predicted LB would be 0.0524

IMPLICATION: We need to CHANGE the CV-LB relationship, not just improve CV

Possible causes of high intercept:
1. Systematic overfitting to training solvents
2. Mismatch between local CV and Kaggle evaluation
3. Distribution shift between train/test
4. Model miscalibration


In [7]:
# What approaches might change the CV-LB relationship?
print('=== APPROACHES TO CHANGE CV-LB RELATIONSHIP ===')
print()
approaches = [
    ('Prediction calibration', 'Adjust predictions to reduce systematic bias'),
    ('Ensemble with constant offset', 'Add a learned offset to predictions'),
    ('Different CV scheme', 'Verify our CV matches Kaggle exactly'),
    ('Regularization tuning', 'Stronger regularization to prevent overfitting'),
    ('Feature selection', 'Remove features that cause overfitting'),
    ('Domain adaptation', 'Explicitly model train-test distribution shift'),
]

for name, desc in approaches:
    print(f'{name}')
    print(f'  {desc}')
    print()

=== APPROACHES TO CHANGE CV-LB RELATIONSHIP ===

Prediction calibration
  Adjust predictions to reduce systematic bias

Ensemble with constant offset
  Add a learned offset to predictions

Different CV scheme
  Verify our CV matches Kaggle exactly

Regularization tuning
  Stronger regularization to prevent overfitting

Feature selection
  Remove features that cause overfitting

Domain adaptation
  Explicitly model train-test distribution shift



In [8]:
# Best experiment analysis
print('=== BEST EXPERIMENT: exp_030 (GP+MLP+LGBM) ===')
print()
print('CV: 0.008298')
print('LB: 0.087720')
print('Gap: 0.0794 (9.6x)')
print()
print('Components:')
print('  GP (0.2): Matern kernel on Spange + Arrhenius (18 features)')
print('  MLP (0.5): [32,16] with weighted loss on full features (145 features)')
print('  LGBM (0.3): Full features (145 features)')
print()
print('Key insight: GP provides different inductive bias')
print('GP may have better uncertainty calibration for OOD solvents')

=== BEST EXPERIMENT: exp_030 (GP+MLP+LGBM) ===

CV: 0.008298
LB: 0.087720
Gap: 0.0794 (9.6x)

Components:
  GP (0.2): Matern kernel on Spange + Arrhenius (18 features)
  MLP (0.5): [32,16] with weighted loss on full features (145 features)
  LGBM (0.3): Full features (145 features)

Key insight: GP provides different inductive bias
GP may have better uncertainty calibration for OOD solvents


In [9]:
# What should we try next?
print('=== RECOMMENDED NEXT STEPS ===')
print()
print('Priority 1: Prediction Calibration')
print('  - Try Platt scaling or isotonic regression on predictions')
print('  - May reduce the intercept in CV-LB relationship')
print()
print('Priority 2: Verify CV Scheme')
print('  - Read template notebook carefully')
print('  - Ensure we leave out full experiments for mixtures')
print('  - Check weighting between single-solvent and mixture data')
print()
print('Priority 3: Stronger Regularization')
print('  - Increase dropout, weight decay')
print('  - Reduce model complexity further')
print('  - May reduce overfitting to training solvents')
print()
print('Priority 4: GP-Heavy Ensemble')
print('  - Increase GP weight (currently 0.2)')
print('  - GP may generalize better to unseen solvents')
print('  - Try pure GP model')

=== RECOMMENDED NEXT STEPS ===

Priority 1: Prediction Calibration
  - Try Platt scaling or isotonic regression on predictions
  - May reduce the intercept in CV-LB relationship

Priority 2: Verify CV Scheme
  - Read template notebook carefully
  - Ensure we leave out full experiments for mixtures
  - Check weighting between single-solvent and mixture data

Priority 3: Stronger Regularization
  - Increase dropout, weight decay
  - Reduce model complexity further
  - May reduce overfitting to training solvents

Priority 4: GP-Heavy Ensemble
  - Increase GP weight (currently 0.2)
  - GP may generalize better to unseen solvents
  - Try pure GP model


In [11]:
# Check the evaluation scheme
import pandas as pd

# Load full data
full_df = pd.read_csv('/home/data/catechol_full_data_yields.csv')
print(f"Full data shape: {full_df.shape}")

# Check unique solvent ramps
solvent_ramps = full_df[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"\nNumber of unique solvent ramps: {len(solvent_ramps)}")
print("\nSolvent ramps:")
for _, row in solvent_ramps.iterrows():
    count = len(full_df[(full_df["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & 
                        (full_df["SOLVENT B NAME"] == row["SOLVENT B NAME"])])
    print(f"  {row['SOLVENT A NAME']} + {row['SOLVENT B NAME']}: {count} samples")

Full data shape: (1227, 19)

Number of unique solvent ramps: 13

Solvent ramps:
  Methanol + Ethylene Glycol [1,2-Ethanediol]: 122 samples
  1,1,1,3,3,3-Hexafluoropropan-2-ol + 2-Methyltetrahydrofuran [2-MeTHF]: 124 samples
  Cyclohexane + IPA [Propan-2-ol]: 104 samples
  Water.Acetonitrile + Acetonitrile: 125 samples
  Acetonitrile + Acetonitrile.Acetic Acid: 125 samples
  2-Methyltetrahydrofuran [2-MeTHF] + Diethyl Ether [Ether]: 124 samples
  2,2,2-Trifluoroethanol + Water.2,2,2-Trifluoroethanol: 125 samples
  DMA [N,N-Dimethylacetamide] + Decanol: 110 samples
  Ethanol + THF [Tetrahydrofuran]: 127 samples
  Dihydrolevoglucosenone (Cyrene) + Ethyl Acetate: 36 samples
  MTBE [tert-Butylmethylether] + Butanone [MEK]: 34 samples
  tert-Butanol [2-Methylpropan-2-ol] + Dimethyl Carbonate: 36 samples
  Methyl Propionate + Ethyl Lactate: 35 samples


In [ ]:
# Final summary
print('=== LOOP 42 SUMMARY ===')
print()
print('ChemBERTa experiment (exp_041):')
print('  - CV = 0.010288 (25.5% WORSE than baseline)')
print('  - Pre-trained molecular embeddings do NOT help')
print('  - Domain-specific Spange descriptors remain superior')
print('  - Correctly decided NOT to submit')
print()
print('Current best:')
print('  - exp_030: CV = 0.008298, LB = 0.087720')
print('  - Target: 0.0347 (2.53x gap)')
print()
print('Remaining submissions: 4')
print()
print('Critical insight:')
print('  - CV-LB intercept (0.0524) > Target (0.0347)')
print('  - Need to change the relationship, not just improve CV')
print('  - Focus on calibration, regularization, or different approach')